In [1]:
import os, glob

import pickle
import numpy as np
import pandas as pd
import lightkurve as lk
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import patches
from tqdm.auto import tqdm

from astropy.io import fits
from astropy.table import Table

path = os.path.dirname(os.getcwd())
sys.path.append(path)
from kepler_apertures import EXBAMachine

%matplotlib inline

In [2]:
Q = 6
quarters = np.arange(5, 18)
channels = np.arange(1, 85)
channels = np.delete(channels, [4,5,6,7])
quarters, channels

(array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
 array([ 1,  2,  3,  4,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
        56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
        73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84]))

In [39]:
for Q in quarters[:4]:
    # Load EXBA stamp, catalo, and metadata
    bls_results = {}
    clcs = {}
    for ch in tqdm(channels, desc="Loading pkls %i" % Q):
        aux = pickle.load(open("../data/export/%i/%02i/bls_results_q%i_ch%02i.pkl" % (Q, ch, Q, ch), "rb"))
        clcs[ch] = aux["lc_corrected"]
        df_ = aux["table"]
        df_["channel"] = ch
        bls_results[ch] = df_

    df = pd.concat(list(bls_results.values()), axis=0)
    df = df.reset_index()
    df.to_csv("../data/export/corrected/%i/bls_shortP_q%i.csv" % (Q, Q))
    
    if True:
        continue
    
    quarter_path = "../data/export/corrected/%i" % (Q)
    for ch, lcs in tqdm(clcs.items(), desc="Writing FITS"):
        channel_path = "%s/%02i" % (quarter_path, ch)
        if not os.path.isdir(channel_path):
            os.mkdir(channel_path)
        for lc in lcs:
            fname = "%s/%s" % (channel_path, lc.meta["FILENAME"].split("/")[-1])
            if True:
                lk.LightCurve(lc).to_fits(path=fname, overwrite=True)


Loading pkls 5:   0%|          | 0/80 [00:00<?, ?it/s]

Loading pkls 6:   0%|          | 0/80 [00:00<?, ?it/s]

Loading pkls 7:   0%|          | 0/80 [00:00<?, ?it/s]

Loading pkls 8:   0%|          | 0/80 [00:00<?, ?it/s]

In [29]:
quarter_path = "../data/export/corrected/%i" % (Q)
for ch, lcs in tqdm(clcs.items()):
    channel_path = "%s/%02i" % (quarter_path, ch)
    if not os.path.isdir(channel_path):
        os.mkdir(channel_path)
    for lc in lcs:
        fname = "%s/%s" % (channel_path, lc.meta["FILENAME"].split("/")[-1])
        if True:
            lk.LightCurve(lc).to_fits(path=fname, overwrite=True)

  0%|          | 0/80 [00:00<?, ?it/s]

# Long BLS

In [3]:
batches = np.arange(1, 95)
batches

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94])

In [4]:
bls_results = []
for batch in tqdm(batches, desc="Batches"):
    aux = pd.read_csv("../data/export/long_bls/bls_longP_results_%02i.csv" % (batch), index_col=0)
    bls_results.append(aux)
    
bls_results = pd.concat(bls_results, axis=0).reset_index()

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

In [5]:
bls_results

,index,designation,BLS_period,BLS_period_SNR,BLS_duration,BLS_depth,BLS_transit_time
0,0,Gaia EDR3 2105384211065299328,348.161331,22.951149,0.450,0.075552,2.455372e+06
1,1,Gaia EDR3 2105384241127278208,348.161331,21.194797,0.450,0.046263,2.455372e+06
2,2,Gaia EDR3 2105384245425035392,348.161331,25.194258,0.450,0.017081,2.455372e+06
3,3,Gaia EDR3 2105384245425036032,350.876304,8.739377,0.330,0.002874,2.455363e+06
4,4,Gaia EDR3 2105384275487021696,304.306366,17.942446,0.030,0.102101,2.455403e+06
...,...,...,...,...,...,...,...
9322,22,Gaia EDR3 2086539951857563136,79.904992,7.173632,0.249,0.062134,2.456065e+06
9323,23,Gaia EDR3 2129526011778662272,86.448762,4.412601,0.030,0.125105,2.456110e+06
9324,24,Gaia EDR3 2143714973114706560,60.000000,3.672957,0.126,0.002612,2.456422e+06
9325,25,Gaia EDR3 2076208704382822912,60.000000,16.093070,0.030,0.016267,2.456395e+06


In [6]:
bls_results.to_csv('../data/export/EXBA_BLS_results_longPeriods_longLCs.csv')